In [16]:
%env PATH=/lab/solexa_weissman/enn/miniconda3/bin:/lab/solexa_weissman/enn/miniconda3/condabin:/usr/lib/jvm/java-8-openjdk-amd64/jre/bin:/usr/local/bin/python3.6:/nfs/lsf8/8.0/linux2.6-glibc2.3-x86_64/etc:/nfs/lsf8/8.0/linux2.6-glibc2.3-x86_64/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/opt/gurobi900/linux64/bin:/snap/bin:/usr/local/genome/bin:/lab/solexa_weissman/enn/miniconda3/envs/genome_mining_env/bin:$PATH

env: PATH=/lab/solexa_weissman/enn/miniconda3/bin:/lab/solexa_weissman/enn/miniconda3/condabin:/usr/lib/jvm/java-8-openjdk-amd64/jre/bin:/usr/local/bin/python3.6:/nfs/lsf8/8.0/linux2.6-glibc2.3-x86_64/etc:/nfs/lsf8/8.0/linux2.6-glibc2.3-x86_64/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/opt/gurobi900/linux64/bin:/snap/bin:/usr/local/genome/bin:/lab/solexa_weissman/enn/miniconda3/envs/genome_mining_env/bin:$PATH


In [42]:
# import packages
import pandas as pd
import gget
import numpy as np
from Bio import SeqIO, AlignIO
from Bio.pairwise2 import align, format_alignment
from Bio.Entrez import efetch
from Bio.Align.Applications import MafftCommandline
from Bio.Align import AlignInfo
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from joblib import Parallel, delayed


In [2]:
# upload the fasta files of the core DNMT3L domains of both mouse and human
# these will be used for the initial BLAST search to find more DNMT3L orthologs
# they will also be used to trim the orthologs down to the core domain

def upload_fasta(fasta_file):
    with open(fasta_file, 'r') as fasta_contents:
        # Read the fasta file into a string
        fasta_contents = fasta_contents.read()

        # Split the contents of the fasta file into a list of lines
        fasta_lines = fasta_contents.split('\n')

        # Remove the header line from the list of lines
        header = fasta_lines[0]
        fasta_lines = fasta_lines[1:]

        # Concatenate the remaining lines into a single string
        fasta_sequence = ''.join(fasta_lines)
    
    return fasta_sequence


HsCore = upload_fasta('DNMT3L_HUMAN_Core.fasta')
MmCore = upload_fasta('Dnmt3l_MOUSE_Core.fasta')


# Print the contents of the fasta file
print(HsCore)
print(MmCore)


NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLKHVVDVTDTVRKDVEEWGPFDLVYGATPPLGHTCDRPPSWYLFQFHRLLQYARPKPGSPRPFFWMFVDNLVLNKEDLDVASRFLEMEPVTIPDVHGGSLQNAVRVWSNIPAIRSRHWALVSEEELSLLAQNKQSSKLAAKWPTKLVKNCFLPLREYFKYFSTELTSSL
GPMEIYKTVSAWKRQPVRVLSLFRNIDKVLKSLGFLESGSGSGGGTLKYVEDVTNVVRRDVEKWGPFDLVYGSTQPLGSSCDRCPGWYMFQFHRILQYALPRQESQRPFFWIFMDNLLLTEDDQETTTRFLQTEAVTLQDVRGRDYQNAMRVWSNIPGLKSKHAPLTPKEEEYLQAQVRSRSKLDAPKVDLLVKNCLLPLREYFKYFSQNSLPL


In [10]:
# BLAST the core DNMT3L sequences
human_blast = gget.blast(HsCore, limit = 1000)

Tue Dec 27 22:00:24 2022 INFO Sequence recognized as amino acid sequence.
Tue Dec 27 22:00:24 2022 INFO BLAST will use program 'blastp' with database 'nr'.
Tue Dec 27 22:00:24 2022 INFO BLAST initiated with search ID URCA8Z0F013. Estimated time to completion: 179 seconds.
Tue Dec 27 22:03:24 2022 INFO BLASTING...
Tue Dec 27 22:04:25 2022 INFO BLASTING...
Tue Dec 27 22:05:26 2022 INFO BLASTING...
Tue Dec 27 22:06:28 2022 INFO BLASTING...
Tue Dec 27 22:07:29 2022 INFO BLASTING...
Tue Dec 27 22:08:30 2022 INFO BLASTING...
Tue Dec 27 22:09:32 2022 INFO BLASTING...
Tue Dec 27 22:10:33 2022 INFO BLASTING...
Tue Dec 27 22:11:34 2022 INFO BLASTING...
Tue Dec 27 22:12:35 2022 INFO BLASTING...
Tue Dec 27 22:13:37 2022 INFO BLASTING...
Tue Dec 27 22:14:47 2022 INFO Retrieving results...


In [13]:
human_blast.to_csv('Hs_3L_BLAST.csv')

In [ ]:
mouse_blast = gget.blast(MmCore, limit = 1000)
mouse_blast.to_csv('Mm_3L_BLAST.csv')

Tue Dec 27 22:21:09 2022 INFO Sequence recognized as amino acid sequence.
Tue Dec 27 22:21:09 2022 INFO BLAST will use program 'blastp' with database 'nr'.
Tue Dec 27 22:21:10 2022 INFO BLAST initiated with search ID URDH6NR6013. Estimated time to completion: 27 seconds.
Tue Dec 27 22:21:37 2022 INFO BLASTING...
Tue Dec 27 22:22:39 2022 INFO BLASTING...
Tue Dec 27 22:23:40 2022 INFO BLASTING...


In [8]:
# import the saved csv files of 1000 BLASTed sequences
# merge the lists from mouse and human and remove duplicates
Hs_BLAST = pd.read_csv('Hs_3L_BLAST.csv')
Mm_BLAST = pd.read_csv('Mm_3L_BLAST.csv')

# Merge the two dataframes and ignore the index of the original dataframes
Merged_BLAST = pd.concat([Hs_BLAST, Mm_BLAST], ignore_index=True)

# Remove duplicate rows based on the 'Accession' column
Merged_BLAST = Merged_BLAST.drop_duplicates(subset='Accession')

#reset the index
Merged_BLAST.reset_index(inplace=True, drop=True)


In [28]:
# Iterate over the rows in the dataframe to add sequence information
for index, row in Merged_BLAST.iterrows():
    # Extract the sequence data for the current row
    accession = row['Accession']
    
    # Search for and retrieve the sequence data for the given accession number
    handle = efetch(id=accession, db='protein', rettype='fasta')

    # Parse the retrieved data
    record = SeqIO.read(handle, 'fasta')

    # Extract the sequence data from the SeqRecord object
    sequence = str(record.seq)

    # Update the data in the dataframe with the trimmed sequence
    Merged_BLAST.at[index, 'Sequence'] = sequence

In [30]:
Merged_BLAST.to_csv('Merged_3L_BLAST.csv')

In [29]:
display(Merged_BLAST)

,Unnamed: 0,Description,Scientific Name,Common Name,Taxid,Max Score,Total Score,Query Cover,E value,Per. Ident,Acc. Len,Accession,Sequence
0,0,Structure of Dnmt3a-Dnmt3L C-terminal domain c...,Homo sapiens,human,9606,437,437,100%,1.000000e-154,100.00%,230,2QRV_B,GSMWRSQLKAFYDRESENPLEMFETVPVWRRQPVRVLSLFEDIKKE...
1,1,DNA (cytosine-5)-methyltransferase 3-like isof...,Homo sapiens,human,9606,441,441,100%,1.000000e-153,100.00%,386,NP_787063.1,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...
2,2,DNA (cytosine-5-)-methyltransferase 3-like [sy...,synthetic construct,NaN,32630,439,439,100%,8.000000e-153,99.53%,386,ABM86298.1,MAAITALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...
3,3,DNA methyltransferase 3 like protein (DNMT3L) ...,Homo sapiens,human,9606,439,439,100%,1.000000e-152,99.53%,386,2PV0_A,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...
4,4,DNA (cytosine-5)-methyltransferase 3-like [Pan...,Pan troglodytes,chimpanzee,9598,437,437,100%,7.000000e-152,99.53%,387,XP_016794127.1,MAAIPALDPEAEPSMDVILVGSSELSSSISPRTGRDLIAYEVKANQ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,994,DNA (cytosine-5)-methyltransferase 3B isoform ...,Panthera leo,lion,9689,105,105,81%,4.000000e-22,30.15%,836,XP_042787195.1,MKGDTRQLNREEDASGREDSIITNGGCSDQSSDSKDAPSPPILEAI...
1440,995,DNA (cytosine-5)-methyltransferase 3B isoform ...,Trichechus manatus latirostris,Florida manatee,127582,105,105,81%,4.000000e-22,30.15%,839,XP_023581094.1,MQETMKGDTRQLNGEEDASRREDSIITNGACSDQSSYSKDAPSPPI...
1441,996,PREDICTED: DNA (cytosine-5)-methyltransferase ...,Odobenus rosmarus divergens,Pacific walrus,9708,105,105,81%,4.000000e-22,30.15%,856,XP_004393101.1,MKGDTRQLNREEDASGREDSIITNGGCSDQSSDSKDAPSPPILEAI...
1442,998,DNA (cytosine-5)-methyltransferase 3A-like iso...,Cyprinodon tularosa,NaN,77115,105,105,82%,4.000000e-22,33.50%,795,XP_038128269.1,MPSNNPAAAEQLQTPENDTSKDVGQDGADQDRPEERLPASPQNKRR...


In [38]:
display(Merged_BLAST)
Merged_BLAST.to_csv('Merged_3L_BLAST.csv')

,Unnamed: 0.1,Unnamed: 0,Description,Scientific Name,Common Name,Taxid,Max Score,Total Score,Query Cover,E value,Per. Ident,Acc. Len,Accession,Sequence,Core Sequence
0,0,0,Structure of Dnmt3a-Dnmt3L C-terminal domain c...,Homo sapiens,human,9606,437,437,100%,1.000000e-154,100.00%,230,2QRV_B,GSMWRSQLKAFYDRESENPLEMFETVPVWRRQPVRVLSLFEDIKKE...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
1,1,1,DNA (cytosine-5)-methyltransferase 3-like isof...,Homo sapiens,human,9606,441,441,100%,1.000000e-153,100.00%,386,NP_787063.1,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
2,2,2,DNA (cytosine-5-)-methyltransferase 3-like [sy...,synthetic construct,NaN,32630,439,439,100%,8.000000e-153,99.53%,386,ABM86298.1,MAAITALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
3,3,3,DNA methyltransferase 3 like protein (DNMT3L) ...,Homo sapiens,human,9606,439,439,100%,1.000000e-152,99.53%,386,2PV0_A,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
4,4,4,DNA (cytosine-5)-methyltransferase 3-like [Pan...,Pan troglodytes,chimpanzee,9598,437,437,100%,7.000000e-152,99.53%,387,XP_016794127.1,MAAIPALDPEAEPSMDVILVGSSELSSSISPRTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,1439,994,DNA (cytosine-5)-methyltransferase 3B isoform ...,Panthera leo,lion,9689,105,105,81%,4.000000e-22,30.15%,836,XP_042787195.1,MKGDTRQLNREEDASGREDSIITNGGCSDQSSDSKDAPSPPILEAI...,EAPKLYPAIPAARRRPIRVLSLFDGIATGYLVLKELGTVKHEGNIK...
1440,1440,995,DNA (cytosine-5)-methyltransferase 3B isoform ...,Trichechus manatus latirostris,Florida manatee,127582,105,105,81%,4.000000e-22,30.15%,839,XP_023581094.1,MQETMKGDTRQLNGEEDASRREDSIITNGACSDQSSYSKDAPSPPI...,MQETMKGDTRQLNGEEDASRREDSIITNGACSDQSSYSKDAPGNIK...
1441,1441,996,PREDICTED: DNA (cytosine-5)-methyltransferase ...,Odobenus rosmarus divergens,Pacific walrus,9708,105,105,81%,4.000000e-22,30.15%,856,XP_004393101.1,MKGDTRQLNREEDASGREDSIITNGGCSDQSSDSKDAPSPPILEAI...,EAPKLYPAIPAARRRPIRVLSLFDGIATGYLVLKELGTVKHEGNIK...
1442,1442,998,DNA (cytosine-5)-methyltransferase 3A-like iso...,Cyprinodon tularosa,NaN,77115,105,105,82%,4.000000e-22,33.50%,795,XP_038128269.1,MPSNNPAAAEQLQTPENDTSKDVGQDGADQDRPEERLPASPQNKRR...,EPAKLYPPVAAEKRKPIRVLSLFDGIATGLLVLKDLGIQVDEGRIM...


In [46]:
# Trim all the amino acid sequences in merged BLAST file to retain only the core domain
# Load the dataframe containing the sequences
Merged_BLAST = pd.read_csv('Merged_3L_BLAST.csv')

# Read in the reference sequence from the fasta file
reference = SeqIO.read('DNMT3L_HUMAN_Core.fasta', 'fasta')

# Create an empty list to store the trimmed sequences
trimmed_sequences = []

# Iterate through the sequences in the dataframe
for index, row in Merged_BLAST.iterrows():
    seq = row['Sequence']

    # Align the reference sequence with the entire target sequence using pairwise2.align.globalxs
    alignment = align.globalxs(reference.seq, seq, -10, -0.5)

    # Extract the aligned sequences from the alignment object
    aligned_seq1 = alignment[0][0]
    aligned_seq2 = alignment[0][1]


    # Trim the sequences to only include the portion that aligns with the core domain
    index_tracker = []
    for i in range(len(aligned_seq1)):
        if aligned_seq1[i] != '-':
            index_tracker.append(i)
    
    trimmed_seq = ''

    # Create list of integers between first and last index
    int_array = np.arange(index_tracker[0], index_tracker[-1]+1)
    int_list = int_array.tolist()
    
    for k in int_list:
        if aligned_seq2[k] != '-':
            trimmed_seq += aligned_seq2[k]
      
    # Add the trimmed sequence to the list
    trimmed_sequences.append(trimmed_seq)

# Add the trimmed sequences to the dataframe as a new column
Merged_BLAST['Core Sequence'] = trimmed_sequences



[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229]
[173, 174, 175, 1

[173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372,

[208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407,

[319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518,

[251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,

...

[499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780,

[499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780,

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881,

[541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822,

[452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781,

In [60]:
Merged_BLAST.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.2'],inplace=True)
display(Merged_BLAST)

,Description,Scientific Name,Common Name,Taxid,Max Score,Total Score,Query Cover,E value,Per. Ident,Acc. Len,Accession,Sequence,Core Sequence
0,Structure of Dnmt3a-Dnmt3L C-terminal domain c...,Homo sapiens,human,9606,437,437,100%,1.000000e-154,100.00%,230,2QRV_B,GSMWRSQLKAFYDRESENPLEMFETVPVWRRQPVRVLSLFEDIKKE...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
1,DNA (cytosine-5)-methyltransferase 3-like isof...,Homo sapiens,human,9606,441,441,100%,1.000000e-153,100.00%,386,NP_787063.1,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
2,DNA (cytosine-5-)-methyltransferase 3-like [sy...,synthetic construct,NaN,32630,439,439,100%,8.000000e-153,99.53%,386,ABM86298.1,MAAITALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
3,DNA methyltransferase 3 like protein (DNMT3L) ...,Homo sapiens,human,9606,439,439,100%,1.000000e-152,99.53%,386,2PV0_A,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
4,DNA (cytosine-5)-methyltransferase 3-like [Pan...,Pan troglodytes,chimpanzee,9598,437,437,100%,7.000000e-152,99.53%,387,XP_016794127.1,MAAIPALDPEAEPSMDVILVGSSELSSSISPRTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,DNA (cytosine-5)-methyltransferase 3B isoform ...,Panthera leo,lion,9689,105,105,81%,4.000000e-22,30.15%,836,XP_042787195.1,MKGDTRQLNREEDASGREDSIITNGGCSDQSSDSKDAPSPPILEAI...,EAPKLYPAIPAARRRPIRVLSLFDGIATGYLVLKELGIKVEKYVAS...
1440,DNA (cytosine-5)-methyltransferase 3B isoform ...,Trichechus manatus latirostris,Florida manatee,127582,105,105,81%,4.000000e-22,30.15%,839,XP_023581094.1,MQETMKGDTRQLNGEEDASRREDSIITNGACSDQSSYSKDAPSPPI...,MQETMKGDTRQLNGEEDASRREDSIITNGACSDQSSYSKDAPSPPI...
1441,PREDICTED: DNA (cytosine-5)-methyltransferase ...,Odobenus rosmarus divergens,Pacific walrus,9708,105,105,81%,4.000000e-22,30.15%,856,XP_004393101.1,MKGDTRQLNREEDASGREDSIITNGGCSDQSSDSKDAPSPPILEAI...,EAPKLYPAIPAARRRPIRVLSLFDGIATGYLVLKELGIKVEKYVAS...
1442,DNA (cytosine-5)-methyltransferase 3A-like iso...,Cyprinodon tularosa,NaN,77115,105,105,82%,4.000000e-22,33.50%,795,XP_038128269.1,MPSNNPAAAEQLQTPENDTSKDVGQDGADQDRPEERLPASPQNKRR...,EPAKLYPPVAAEKRKPIRVLSLFDGIATGLLVLKDLGIQVDKYVAS...


In [61]:
Merged_BLAST.to_csv('Merged_3L_BLAST.csv')

In [65]:
# Iterate over the rows in the dataframe to add sequence information
for index, row in Merged_BLAST.iterrows():
    # Extract the sequence data for the current row
    core_seq = row['Core Sequence']
    length = len(core_seq)

    # Update the data in the dataframe with the trimmed sequence
    Merged_BLAST.at[index, 'Length'] = length

# Filter the dataframe and keep only the rows where the "Length" column is between 180 and 250 (inclusive)
Merged_BLAST_filtered = Merged_BLAST.query('180 <= Length <= 250')
Merged_BLAST_filtered.reset_index(inplace=True, drop=True)
display(Merged_BLAST_filtered)

,Description,Scientific Name,Common Name,Taxid,Max Score,Total Score,Query Cover,E value,Per. Ident,Acc. Len,Accession,Sequence,Core Sequence,Length
0,Structure of Dnmt3a-Dnmt3L C-terminal domain c...,Homo sapiens,human,9606,437,437,100%,1.000000e-154,100.00%,230,2QRV_B,GSMWRSQLKAFYDRESENPLEMFETVPVWRRQPVRVLSLFEDIKKE...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...,213.0
1,DNA (cytosine-5)-methyltransferase 3-like isof...,Homo sapiens,human,9606,441,441,100%,1.000000e-153,100.00%,386,NP_787063.1,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...,213.0
2,DNA (cytosine-5-)-methyltransferase 3-like [sy...,synthetic construct,NaN,32630,439,439,100%,8.000000e-153,99.53%,386,ABM86298.1,MAAITALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...,213.0
3,DNA methyltransferase 3 like protein (DNMT3L) ...,Homo sapiens,human,9606,439,439,100%,1.000000e-152,99.53%,386,2PV0_A,MAAIPALDPEAEPSMDVILVGSSELSSSVSPGTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...,213.0
4,DNA (cytosine-5)-methyltransferase 3-like [Pan...,Pan troglodytes,chimpanzee,9598,437,437,100%,7.000000e-152,99.53%,387,XP_016794127.1,MAAIPALDPEAEPSMDVILVGSSELSSSISPRTGRDLIAYEVKANQ...,NPLEMFETVPVWRRQPVRVLSLFEDIKKELTSLGFLESGSDPGQLK...,214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,DNA (cytosine-5)-methyltransferase 3B-like iso...,Carcharodon carcharias,great white shark,13397,108,108,94%,6.000000e-23,29.26%,756,XP_041060806.1,MDSEQERLTMKSNDCNSETIGNISKLQPVVNGEKTYQLDQWEQATP...,DPPKIYPAVPAEHRRPIKVLSLFDGIATGYLVLRDLGFKVERYIAS...,232.0
617,DNA (cytosine-5)-methyltransferase 3B isoform ...,Prionailurus viverrinus,fishing cat,61388,107,107,74%,1.000000e-22,32.97%,768,XP_047708055.1,MCRVKILRSARYSPLFACRPASPLLRLADPCPYQETMKGDTRQLNR...,MCRVKILRSARYSPLFACRPASPLLRLADPCPYQETMKGDTRQLNR...,213.0
618,hypothetical protein Y1Q_0020292 [Alligator mi...,Alligator mississippiensis,American alligator,8496,106,106,72%,2.000000e-22,32.60%,795,KYO24181.1,MTGSRRWEGPAEASSTNPAPKASMKKENPPSGEMDCRAELVFLSAG...,EAPKIYPAIPAAKRKPIRVLSLFDGIATGYLVLKDLGIKVEKYIAS...,227.0
619,DNA (cytosine-5)-methyltransferase 3B isoform ...,Prionailurus viverrinus,fishing cat,61388,106,106,73%,2.000000e-22,31.91%,763,XP_047708057.1,MCRVKILRSARYSPLFACRPASPLLRLADPCPYQETMKGDTRQLNR...,MCRVKILRSARYSPLFACRPASPLLRLADPCPYQETMKGDTRQLNR...,213.0


In [66]:
Merged_BLAST_filtered.to_csv('Merged_3L_BLAST_filtered.csv')

In [18]:
Merged_BLAST_filtered = pd.read_csv('Merged_3L_BLAST_filtered.csv')

In [19]:
# Find the maximum length of the strings in the "Core Sequence" column
max_length = Merged_BLAST_filtered["Core Sequence"].str.len().max()

# Pad the strings in the "Core Sequence" column with "-" characters on the right side
Merged_BLAST_filtered["Core Sequence"] = Merged_BLAST_filtered["Core Sequence"].str.pad(max_length, side="right", fillchar="-")

# Verify that the strings in the "Core Sequence" column are now all the same length
assert Merged_BLAST_filtered["Core Sequence"].str.len().max() == max_length

In [20]:
df_sample = Merged_BLAST_filtered.head(10)

In [5]:
# get pairwise distance matrix between all the sequences using MAFFT alignment

# Create an empty identity matrix
num_sequences = Merged_BLAST_filtered.shape[0]
identity_matrix = np.zeros((num_sequences, num_sequences))

# Iterate through the rows of the dataframe
with Parallel(n_jobs=1000) as parallel:
    for i, row1 in Merged_BLAST_filtered.iterrows():
        for j, row2 in Merged_BLAST_filtered.iterrows():
            if i >= j:
                continue  # skip self-comparisons and duplicates
            # Align the sequences using MAFFT
            mafft_cline = MafftCommandline(input="temp_in.fasta", auto=True)
            with open("temp_in.fasta", "w") as f:
                SeqIO.write([SeqRecord(Seq(row1["Core Sequence"]), id=str(i)), SeqRecord(Seq(row2["Core Sequence"]), id=str(j))], f, "fasta")
            stdout, stderr = mafft_cline()
            alignment = AlignIO.read("temp_in.fasta", "fasta")

            # Calculate the percent identity
            identity = 0
            for k in range(len(alignment[0,:])):
                if alignment[0,k] == alignment[1,k]:
                    identity += 1
            percent_identity = identity / len(alignment[0,:]) * 100

            # Add the percent identity to the identity matrix
            identity_matrix[i, j] = percent_identity
            identity_matrix[j, i] = percent_identity
            
            print(identity_matrix)

            # Convert the identity matrix to a Pandas DataFrame
            identity_df = pd.DataFrame(identity_matrix)

            # Save the DataFrame as a CSV file
            identity_df.to_csv("identity_matrix.csv")
            
        print("completed row ",i)
        


[[  0. 100.   0. ...   0.   0.   0.]
 [100.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ... 

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

...

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.    0.    0. ]]
[[  0.  100.   99.6 ...   0.    0.    0. ]
 [100.    0.    0.  ...   0.    0.    0. ]
 [ 99.6   0.    0.  ...   0.    0.    0. ]
 ...
 [  0.    0.    0.  ...   0.    0.    0. ]
 [  0.    0.    0.  ...   0.   

KeyboardInterrupt: 

In [52]:
# define a function to do the pairwise distances
def pairwise_dist(i,dataframe):
    # Create an empty identity matrix
    num_sequences = dataframe.shape[0]
    identity_matrix = np.zeros((num_sequences, num_sequences))
    df_list = dataframe['Core Sequence'].tolist()
    
    # get first sequence
    seq1 = df_list[i]

    for j in range(len(df_list)):
        if i >= j:
            continue  # skip self-comparisons and duplicates
        seq2 = df_list[j]
        # Align the reference sequence with the target sequence using pairwise2.align.globalxs
        alignment = align.globalxs(seq1, seq2, -10, -0.5)

        # Extract the aligned sequences from the alignment object
        aligned_seq1 = alignment[0][0]
        aligned_seq2 = alignment[0][1]

        # Trim the sequences to only include the portion that aligns with the core domain
        aligned_len = len(aligned_seq1)
        id_tracker = 0
        for k in range(aligned_len):
            if aligned_seq1[k] == aligned_seq2[k]:
                id_tracker += 1
        percent_identity = id_tracker / aligned_len * 100
        # Add the percent identity to the identity matrix
        identity_matrix[i, j] = percent_identity
        identity_matrix[j, i] = percent_identity

    return identity_matrix
    


In [45]:
df_sample_list = df_sample['Core Sequence'].tolist()
seq1 = df_sample_list[0]
seq2 = df_sample_list[8]

# Align the reference sequence with the entire target sequence using pairwise2.align.globalxs
alignment = align.globalxs(seq1, seq2, -10, -0.5)

# Extract the aligned sequences from the alignment object
aligned_seq1 = alignment[0][0]
aligned_seq2 = alignment[0][1]


# Trim the sequences to only include the portion that aligns with the core domain
aligned_len = len(aligned_seq1)
id_tracker = 0
for i in range(aligned_len):
    if aligned_seq1[i] == aligned_seq2[i]:
        id_tracker += 1
identity = id_tracker / aligned_len * 100
print(identity)

98.03149606299213


In [53]:
# initiate the parallel process
identity_matrix = []
identity_matrix = Parallel(n_jobs=2)(delayed(pairwise_dist)(i, df_sample) for i in range(len(df_sample_list)))

/lab/solexa_weissman/enn/miniconda3/envs/genome_mining_env/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/lab/solexa_weissman/enn/miniconda3/envs/genome_mining_env/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [54]:
print(identity_matrix)

[array([[  0.        , 100.        ,  99.6       ,  99.6       ,
         99.60159363,  99.60159363,  98.42519685,  99.20318725,
         98.03149606,  99.20318725],
       [100.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 99.6       ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 99.6       ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 99.60159363,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 99.60159363,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ 

In [55]:
# concatenate output
num_sequences = df_sample.shape[0]
identity_matrix_concat = np.zeros((num_sequences, num_sequences))

for n in range(len(identity_matrix)):
    identity_matrix_concat = identity_matrix_concat + identity_matrix[n]
    
print(identity_matrix_concat)

[[  0.         100.          99.6         99.6         99.60159363
   99.60159363  98.42519685  99.20318725  98.03149606  99.20318725]
 [100.           0.          99.6         99.6         99.60159363
   99.60159363  98.42519685  99.20318725  98.03149606  99.20318725]
 [ 99.6         99.6          0.         100.          99.20318725
   99.20318725  98.03149606  98.80478088  97.63779528  99.60159363]
 [ 99.6         99.6        100.           0.          99.20318725
   99.20318725  98.03149606  98.80478088  97.63779528  99.60159363]
 [ 99.60159363  99.60159363  99.20318725  99.20318725   0.
  100.          98.03921569  99.6         97.64705882  99.6       ]
 [ 99.60159363  99.60159363  99.20318725  99.20318725 100.
    0.          98.03921569  99.6         97.64705882  99.6       ]
 [ 98.42519685  98.42519685  98.03149606  98.03149606  98.03921569
   98.03921569   0.          97.64705882  99.6         97.64705882]
 [ 99.20318725  99.20318725  98.80478088  98.80478088  99.6
   99.6    

In [57]:
# Convert the identity matrix to a Pandas DataFrame
identity_df = pd.DataFrame(identity_matrix_concat)

# Save the DataFrame as a CSV file
identity_df.to_csv("identity_matrix.csv")